In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

In [2]:
# stocks data csv read
df = pd.read_csv('data.csv')
df = df.set_index('Date')

# s&p data csv read
df_sp = pd.read_csv('sp500.csv')
df_sp = df_sp.set_index('Date')

In [3]:
def date_slicer(df, start, duration, rebalancing_period=0):
    start = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=rebalancing_period))
    end = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=duration) - rd(days=1))
    return df.loc[start:end]

In [4]:
def data_process(df):
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    df = df.to_numpy()
    df = torch.from_numpy(df).type(torch.Tensor)
    return df

In [5]:
def daily_change(df):
    df = df.pct_change()
    df = df.tail(-1)
    return df

In [6]:
def daily_return(df):
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    return df

In [7]:
# shallow nnf biuld
class shallow_NNF(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [8]:
# deep nnf build
class deep_NNF(nn.Module):
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [9]:
# 1/N model build
class equal_w_model():
    def __init__(self, df):
        self.df = df
        self.performance()
        
    def performance(self):
        self.df = np.array(self.df)
        weights = np.ones((len(self.df), 1)) * (1/len(self.df))
        cumulative_change = sum(np.multiply(weights, self.df.reshape(-1,1)))
        return cumulative_change, weights.reshape(-1)

In [10]:
# rebalancing period = one or three months
rbp = 3

# epochs
num_epochs = 10

In [11]:
# shallow_nnf hyperparameters
input_dim = 471
hidden_size = 471
num_classes = 471
lr = 1e-3

In [12]:
# shallow nnf tune
shallow_NNF = shallow_NNF(input_dim=input_dim, hidden_size=hidden_size, num_classes=num_classes)
shallow_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_optimizer = torch.optim.Adam(shallow_NNF.parameters(), lr=lr)

In [13]:
# deep_nnf hyperparameters
input_dim = 471
hidden_size1 = 471
hidden_size2 = 471
hidden_size3 = 471
hidden_size4 = 471
hidden_size5 = 471
num_classes = 471
lr = 1e-10
dropout_p = 0.5

In [14]:
# deep nnf tune
deep_NNF = deep_NNF(input_dim=input_dim, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=num_classes)
deep_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_optimizer = torch.optim.Adam(deep_NNF.parameters(), lr=lr)

In [15]:
# RMSE
def RMSE(x, y, weights):
    temp = 0
    for i in range(len(x)):
        temp += (sum(x.iloc[i] * weights) - y.iloc[i]) ** 2
    return math.sqrt(temp/len(x))

In [16]:
# MEAN
def MEAN(x, weights):
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.mean()

In [17]:
# Volatility
def VOL(x, weights):
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.std()

In [18]:
def valid_fun(x_valid, i, model):
    x_change = daily_change(date_slicer(df, '2017-07-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2017-07-01', 6, i))
    # x_return = daily_return(date_slicer(df, '2017-07-01', 6, i))
    # y_return = daily_return(date_slicer(df_sp, '2017-07-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_valid).performance()[1]
    else:
        weights = np.array(model(x_valid)[1].detach())
    
    valid_rmse = RMSE(x_change, y_change, weights)
    # valid_mean = MEAN(x_return, weights)
    # valid_vol  = VOL(x_return, weights)
    
    print(f'Validation RMSE: {valid_rmse}')
    # print(f'Validation MEAN: {valid_mean}')
    # print(f'Validation VOL: {valid_vol}')
    
    return valid_rmse

In [19]:
def test_fun(x_test, i, model):
    x_change = daily_change(date_slicer(df, '2018-01-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2018-01-01', 6, i))
    x_return = daily_return(date_slicer(df, '2018-01-01', 6, i))
    y_return = daily_return(date_slicer(df_sp, '2018-01-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    test_rmse = RMSE(x_change, y_change, weights)
    test_mean = MEAN(x_return, weights)
    test_vol  = VOL(x_return, weights)
    test_dic = {'RMSE': test_rmse, 'MEAN': test_mean, 'VOL': test_vol}
    
    print(f'Test RMSE: {test_rmse}')
    print(f'Test MEAN: {test_mean}')
    print(f'Test VOL: {test_vol}')
    
    return test_dic

### **Deep NNF Training**

In [20]:
# deep nnf training function
def train_deep_nnf(x_train, y_train, i):
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF(x_train)[0]
        loss_deep_nnf = deep_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [21]:
# deep nnf
deep_nnf_valid_rmse_list = []
deep_nnf_test_results = []

for i in range(int(24/rbp)):
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    train_deep_nnf(x_train, y_train, i*rbp)
    deep_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, deep_NNF))
    deep_nnf_test_results.append(test_fun(x_test, i*rbp, deep_NNF))
    deep_NNF.reset_parameters()

print(f'\nMin Valid RMSE is: {min(deep_nnf_valid_rmse_list)} for model i = {deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))+1}')
print('Selected Model Test Results are:')
print('RMSE =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['RMSE'])
print('MEAN =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['MEAN'])
print('VOL =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['VOL'])

deep_best_result_index = deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))


Deep NNF Training & Results for model 1.0:
Epoch 1 of 10 | MSE: 0.0453510545194149
Epoch 10 of 10 | MSE: 0.04540720209479332
Training time: 0.12
Validation RMSE: 0.0013739499998851896
Test RMSE: 0.00168847468285762
Test MEAN: 1.0002240249659242
Test VOL: 0.009413433747836862


/Users/hamidrezarahimzadeh/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Deep NNF Training & Results for model 2.0:
Epoch 1 of 10 | MSE: 0.05991291627287865
Epoch 10 of 10 | MSE: 0.05992415174841881
Training time: 0.09
Validation RMSE: 0.0017827696082611218
Test RMSE: 0.0017376562465409647
Test MEAN: 1.0008831266757166
Test VOL: 0.005777926402437148

Deep NNF Training & Results for model 3.0:
Epoch 1 of 10 | MSE: 0.03910521790385246
Epoch 10 of 10 | MSE: 0.03906742483377457
Training time: 0.08
Validation RMSE: 0.0016832860497979054
Test RMSE: 0.002381482561318416
Test MEAN: 0.9993485500054662
Test VOL: 0.01030185734040395

Deep NNF Training & Results for model 4.0:
Epoch 1 of 10 | MSE: 0.031516917049884796
Epoch 10 of 10 | MSE: 0.03145839646458626
Training time: 0.08
Validation RMSE: 0.0017408396424157281
Test RMSE: 0.0023014829543107724
Test MEAN: 1.0000715658757815
Test VOL: 0.011513002277305682

Deep NNF Training & Results for model 5.0:
Epoch 1 of 10 | MSE: 0.0354909673333168
Epoch 10 of 10 | MSE: 0.03554745018482208
Training time: 0.07
Validation RMSE

### **Shallow NNF Training**

In [22]:
# shallow nnf training function
def train_shallow_nnf(x_train, y_train, i):
    start_time_shallow_nnf = time.time()
    print(f'\nShallow NNF Training & Results for model {(i/rbp)+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF(x_train)[0]
        loss_shallow_nnf = shallow_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [23]:
#shallow nnf
shallow_nnf_valid_rmse_list = []
shallow_nnf_test_results = []

for i in range(int(24/rbp)):
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    
    train_shallow_nnf(x_train, y_train, i*rbp)
    shallow_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, shallow_NNF))
    shallow_nnf_test_results.append(test_fun(x_test, i*rbp, shallow_NNF))
    shallow_NNF.reset_parameters()

# print(f'\nMin Valid RMSE is: {min(valid_rmse_list)} for model i = {(deep_best_result_index)+1}')
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', shallow_nnf_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', shallow_nnf_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', shallow_nnf_test_results[(deep_best_result_index)]['VOL'])


Shallow NNF Training & Results for model 1.0:
Epoch 1 of 10 | MSE: 0.04361874982714653
Epoch 10 of 10 | MSE: 0.0003051921376027167
Training time: 0.05
Validation RMSE: 0.0014437985776871087
Test RMSE: 0.0017150811135603508
Test MEAN: 1.0002104457069043
Test VOL: 0.009401962074030877

Shallow NNF Training & Results for model 2.0:
Epoch 1 of 10 | MSE: 0.05850591883063316
Epoch 10 of 10 | MSE: 0.006951252464205027
Training time: 0.04
Validation RMSE: 0.0018643285918212098
Test RMSE: 0.001771185146058416
Test MEAN: 1.0008802836402695
Test VOL: 0.00576386000210284

Shallow NNF Training & Results for model 3.0:
Epoch 1 of 10 | MSE: 0.0388982892036438
Epoch 10 of 10 | MSE: 0.005437067709863186
Training time: 0.05
Validation RMSE: 0.001747951130277338
Test RMSE: 0.002443211037178915
Test MEAN: 0.999358731647983
Test VOL: 0.01024032912509717

Shallow NNF Training & Results for model 4.0:
Epoch 1 of 10 | MSE: 0.035032059997320175
Epoch 10 of 10 | MSE: 0.004520662594586611
Training time: 0.05
Va

### **1/N Model**

In [26]:
equal_w_model_valid_rmse_list = []
equal_w_model_test_results = []

for i in range(int(24/rbp)):
    print(f'\nEqual Weights Model Results for model {i+1}:')
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    
    equal_w_model_valid_rmse_list.append(valid_fun(x_valid, i*rbp, equal_w_model))
    equal_w_model_test_results.append(test_fun(x_test, i*rbp, equal_w_model))
    
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', equal_w_model_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', equal_w_model_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', equal_w_model_test_results[(deep_best_result_index)]['VOL'])


Equal Weights Model Results for model 1:
Validation RMSE: 0.0013752466607634818
Test RMSE: 0.0016838896697022906
Test MEAN: 1.000223979363228
Test VOL: 0.009419904643453112

Equal Weights Model Results for model 2:
Validation RMSE: 0.00178815568268382
Test RMSE: 0.0017472761704575702
Test MEAN: 1.0008824136684178
Test VOL: 0.005771463367467524

Equal Weights Model Results for model 3:
Validation RMSE: 0.0016838896697022906
Test RMSE: 0.002378854806423318
Test MEAN: 0.9993495431539628
Test VOL: 0.01030115605112

Equal Weights Model Results for model 4:
Validation RMSE: 0.0017472761704575702
Test RMSE: 0.0023085543362458107
Test MEAN: 1.0000718893313152
Test VOL: 0.011509377237775696

Equal Weights Model Results for model 5:
Validation RMSE: 0.002378854806423318
Test RMSE: 0.0019686564797122682
Test MEAN: 1.001545688626488
Test VOL: 0.00785354280865888

Equal Weights Model Results for model 6:
Validation RMSE: 0.0023085543362458107
Test RMSE: 0.0019462210070632896
Test MEAN: 1.000396900

In [25]:
# print test results
print(f'Models test results with rebalancing period of {rbp} month(s) are: ')
deep_temp = pd.DataFrame(deep_nnf_test_results)
deep_temp = deep_temp.iloc[deep_best_result_index]
shallow_temp = pd.DataFrame(shallow_nnf_test_results)
shallow_temp = shallow_temp.iloc[deep_best_result_index]
equal_w_temp = pd.DataFrame(equal_w_model_test_results)
equal_w_temp = equal_w_temp.iloc[deep_best_result_index]

sp_temp_rmse = '-'
sp_temp_mean = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).mean()[0]
sp_temp_std = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).std()[0]
sp_temp = pd.DataFrame([sp_temp_rmse, sp_temp_mean, sp_temp_std], index=deep_temp.index)

final_result = pd.concat([deep_temp, shallow_temp, equal_w_temp, sp_temp], axis=1, join='inner')
final_result.columns = ['Deep NNF', 'Shallow NNF', '1/N Model', 'S&P 500']
final_result

Models test results with rebalancing period of 3 month(s) are: 


,Deep NNF,Shallow NNF,1/N Model,S&P 500
RMSE,0.001688,0.001715,0.001684,-
MEAN,1.000224,1.000210,1.000224,1.000121
VOL,0.009413,0.009402,0.009420,0.010367
